<a href="https://colab.research.google.com/github/sanviaithal26/Architecting-LLMs-WiDS/blob/main/week3/week3_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#E01: Hyperparameters
1.   Block size
2.   Embedding Size
1.   Number of Neurons in the hidden layer
2.   Batch Size
1.   Learning Rate Decay
2.   Total Iterations







In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [7]:
# read in all the words
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
words = open('names.txt', 'r').read().splitlines()
words[:8]

--2026-01-01 05:28:20--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.03s   

2026-01-01 05:28:21 (8.38 MB/s) - ‘names.txt’ saved [228145/228145]



['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [8]:
len(words)

32033

In [9]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [11]:
# ------------ now made respectable :) ---------------

In [12]:
Xtr.shape, Ytr.shape # dataset

(torch.Size([182625, 3]), torch.Size([182625]))

In [13]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
n_hidden = 200                                # 250 hidden neurons
C = torch.randn((27, 16), generator=g)        # embedding space of 36
W1 = torch.randn((48, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = torch.randn((n_hidden, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [14]:
sum(p.nelement() for p in parameters) # number of parameters in total

15659

In [15]:
for p in parameters:
  p.requires_grad = True

In [16]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [17]:
lri = []
lossi = []
stepi = []

In [18]:
batch_size = 16
iterations = 200000
for i in range(iterations):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))

  # forward pass
  emb = C[Xtr[ix]] # (128, 4, 36)
  h = torch.tanh(emb.view(-1, 48) @ W1 + b1) # (128, 250)
  logits = h @ W2 + b2 # (128, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

#print(loss.item())

In [19]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 48) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.1358, grad_fn=<NllLossBackward0>)

In [20]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 48) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1784, grad_fn=<NllLossBackward0>)

Batch size = 4, Embedding = 36, Neurons =250, Batch = 128, Total Iterations = 10000, -----> Overfitted ____________ Training, validation --- 2.02, 2.18

Batch size = 4, Embedding = 36, Neurons =250, Batch = 64, Total Iterations = 10000, -----> Even Worse Overfitting

Batch size = 4, Embedding = 16, Neurons =250, Batch = 16, Total Iterations = 10000, -----> Very Little Overfitting, But high loss value --- 2.47/2.49

Batch size = 3, Embedding = 16, Neurons =200, Batch = 48, Total Iterations = 20000, -----> Not Much Overfitting, good loss value --- 2.08/2.14

### Batch size = **3**, Embedding = **16**, Neurons =**200**, Batch = **16**, Total Iterations = **20000**, -----> Not Much Overfitting, Best loss value --- **2.0572/2.1300**

#E02: Initialisation
If the predicted probabilities at initialization were perfectly uniform, then the probability of predicting the next character would be 1/27 for each character. This means that the probability would be -ln(1/27) = 3.2958

But the actual predicted loss is around 27, which means the model is confidently wrong. This is because the values randomly assigned to W and b matrices are "too random". This loss drops rapidly to near 3 and then the loss gradually reduces. This jump can be avoided by initializing the random values in a normal distribution instead of a truly random pattern.

In [26]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
class Linear:

  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.zeros((fan_in, fan_out), requires_grad=True)
    #self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out, requires_grad=True) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim, requires_grad=True)
    self.beta = torch.zeros(dim, requires_grad=True)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility
vocab_size =27
block_size = 3

C = torch.randn((vocab_size, n_embd), generator=g, requires_grad=True)

layers = [
  Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]
# layers = [
#   Linear(n_embd * block_size, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, n_hidden), Tanh(),
#   Linear(           n_hidden, vocab_size),
# ]

with torch.no_grad():
  # last layer: make less confident
  layers[-1].gamma *= 0.1
  #layers[-1].weight *= 0.1
  # all other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 1.0 #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

47024


In [27]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  x = emb.view(emb.shape[0], -1) # concatenate the vectors
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function

  for p in parameters:
        p.grad = None # Clear old gradients
  loss.backward()

  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
      p.data += -lr * p.grad


  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

      0/ 200000: 3.2958
  10000/ 200000: 2.7882
  20000/ 200000: 2.7518
  30000/ 200000: 2.8506
  40000/ 200000: 2.8221
  50000/ 200000: 2.8987
  60000/ 200000: 2.6038
  70000/ 200000: 2.6751
  80000/ 200000: 3.0131
  90000/ 200000: 2.7753
 100000/ 200000: 2.6079
 110000/ 200000: 2.5352
 120000/ 200000: 2.6702
 130000/ 200000: 2.6442
 140000/ 200000: 2.8332
 150000/ 200000: 2.5482
 160000/ 200000: 2.9762
 170000/ 200000: 2.9521
 180000/ 200000: 2.8626
 190000/ 200000: 2.8505


##Initialising W1 b1 to zeros ^^:

If weights and biases are initialised to zero, the network enters a state of symmetry, preventing it from learning correctly. Because every hidden neuron starts at zero, receives the same input, and produces identical tanh(0)=0 activations, they all receive identical gradients. The entire hidden layer (e.g., 200 neurons) behaves like a single neuron. However, the Embedding Layer (C) and Output Layer (W2, b2) can still differentiate because they have their own random initializations.

It is seen above that the loss moves randomly and does not decrease as it should be.


##BatchNorm 3-layer MLP


In [28]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

class Linear:
  def __init__(self, fan_in, fan_out, bias=True):
    # Kaiming Initialization: scale by 1/sqrt(fan_in)
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out

  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    self.gamma = torch.ones(dim) # Scale parameter
    self.beta = torch.zeros(dim)  # Shift parameter
    self.running_mean = torch.zeros(dim) # Buffer
    self.running_var = torch.ones(dim)   # Buffer

  def __call__(self, x):
    if self.training:
      xmean = x.mean(0, keepdim=True)
      xvar = x.var(0, keepdim=True)
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # Normalize
    self.out = self.gamma * xhat + self.beta
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

In [29]:
# Set to eval mode to use running buffers
for layer in layers:
    if isinstance(layer, BatchNorm1d):
        layer.training = False

# 1. Extract original parameters
W = layers[0].weight
b = torch.zeros(W.shape[1]) # b was None because bias=False

# 2. Extract BatchNorm parameters
gamma = layers[1].gamma
beta = layers[1].beta
mu = layers[1].running_mean
var = layers[1].running_var
eps = layers[1].eps

# 3. Calculate folded weight and bias
scale = gamma / torch.sqrt(var + eps)
W_folded = W * scale
b_folded = beta + (b - mu) * scale

# 4. Verify the forward pass
# Original forward path
x_test = emb.view(emb.shape[0], -1)
out_original = layers[1](layers[0](x_test))

# Fused forward path
out_fused = x_test @ W_folded + b_folded

# Check closeness
max_diff = (out_original - out_fused).abs().max().item()
print(f"Folding successful! Max difference: {max_diff:.4e}")

Folding successful! Max difference: 0.0000e+00
